# Día 3: Funciones Limpias\n\n## Descripción General\n\nLas funciones son los bloques fundamentales de construcción en la programación. Escribir funciones limpias y bien diseñadas es esencial para crear código mantenible, legible y fácil de probar. En este notebook, aprenderás los principios clave para escribir funciones que sean pequeñas, enfocadas y fáciles de entender.\n\nUna función limpia hace una cosa y la hace bien. Tiene un nombre descriptivo, pocos parámetros, y es fácil de leer de arriba a abajo como una narrativa.

## Objetivos de Aprendizaje\n\nAl finalizar este notebook, serás capaz de:\n\n1. Aplicar el principio de responsabilidad única (SRP) a funciones\n2. Escribir funciones pequeñas y enfocadas que hagan una sola cosa\n3. Limitar el número de parámetros en funciones para mejorar la legibilidad\n4. Identificar y refactorizar funciones que hacen demasiado\n5. Organizar código en niveles de abstracción apropiados

## 1. El Principio de Responsabilidad Única (SRP)\n\n### El Problema que Resuelve\n\nCuando una función intenta hacer múltiples cosas, se vuelve difícil de entender, probar y mantener. Los cambios en una parte de la funcionalidad pueden afectar inadvertidamente otras partes.

In [ ]:
# BAD: Function doing too many things\ndef process_user_data(user_dict):\n    \"\"\"Process user data - validates, transforms, saves, and sends email.\"\"\"\n    # Validate\n    if not user_dict.get('email'):\n        raise ValueError('Email required')\n    if '@' not in user_dict['email']:\n        raise ValueError('Invalid email')\n    \n    # Transform\n    user_dict['email'] = user_dict['email'].lower().strip()\n    user_dict['name'] = user_dict.get('name', '').title()\n    \n    # Save to database\n    db_connection = get_database_connection()\n    db_connection.execute('INSERT INTO users VALUES (?)', user_dict)\n    \n    # Send welcome email\n    email_client = get_email_client()\n    email_client.send(user_dict['email'], 'Welcome!')\n    \n    return user_dict

In [ ]:
# GOOD: Each function has a single responsibility\ndef validate_user_email(email: str) -> str:\n    \"\"\"\n    Validate and normalize email address.\n    \n    :param email: Email address to validate\n    :type email: str\n    :return: Normalized email address\n    :rtype: str\n    :raises ValueError: If email is invalid\n    \"\"\"\n    if not email:\n        raise ValueError('Email required')\n    if '@' not in email:\n        raise ValueError('Invalid email')\n    return email.lower().strip()\n\n\ndef normalize_user_name(name: str) -> str:\n    \"\"\"\n    Normalize user name to title case.\n    \n    :param name: User name\n    :type name: str\n    :return: Normalized name\n    :rtype: str\n    \"\"\"\n    return name.title() if name else ''\n\n\ndef save_user_to_database(user_data: dict) -> None:\n    \"\"\"\n    Save user data to database.\n    \n    :param user_data: User information\n    :type user_data: dict\n    \"\"\"\n    db_connection = get_database_connection()\n    db_connection.execute('INSERT INTO users VALUES (?)', user_data)\n\n\ndef send_welcome_email(email: str) -> None:\n    \"\"\"\n    Send welcome email to new user.\n    \n    :param email: User email address\n    :type email: str\n    \"\"\"\n    email_client = get_email_client()\n    email_client.send(email, 'Welcome!')\n\n\ndef process_user_data(user_dict: dict) -> dict:\n    \"\"\"\n    Process new user registration.\n    \n    :param user_dict: Raw user data\n    :type user_dict: dict\n    :return: Processed user data\n    :rtype: dict\n    \"\"\"\n    user_dict['email'] = validate_user_email(user_dict.get('email', ''))\n    user_dict['name'] = normalize_user_name(user_dict.get('name', ''))\n    save_user_to_database(user_dict)\n    send_welcome_email(user_dict['email'])\n    return user_dict

### Aprendizaje Clave\n\nUna función debe hacer una cosa, hacerla bien, y ser la única que la haga. Si puedes describir lo que hace una función con \"y\" o \"o\", probablemente está haciendo demasiado.\n\n**Referencia oficial:** [PEP 8 - Style Guide for Python Code](https://peps.python.org/pep-0008/)

## 2. Funciones Pequeñas\n\n### El Problema que Resuelve\n\nLas funciones largas son difíciles de entender, probar y mantener. Requieren que el lector mantenga mucho contexto en su mente.

In [ ]:
# BAD: Long function with multiple levels of abstraction\ndef generate_report(data):\n    \"\"\"Generate a comprehensive report from data.\"\"\"\n    # Parse data\n    parsed = []\n    for item in data:\n        if item.get('valid'):\n            parsed.append({\n                'id': item['id'],\n                'value': float(item['value']),\n                'category': item.get('category', 'unknown')\n            })\n    \n    # Calculate statistics\n    total = sum(item['value'] for item in parsed)\n    average = total / len(parsed) if parsed else 0\n    by_category = {}\n    for item in parsed:\n        cat = item['category']\n        if cat not in by_category:\n            by_category[cat] = []\n        by_category[cat].append(item['value'])\n    \n    # Format output\n    report = f\"Total: {total}\\n\"\n    report += f\"Average: {average:.2f}\\n\"\n    report += \"\\nBy Category:\\n\"\n    for cat, values in by_category.items():\n        cat_avg = sum(values) / len(values)\n        report += f\"  {cat}: {cat_avg:.2f}\\n\"\n    \n    return report

In [ ]:
# GOOD: Small, focused functions\ndef parse_valid_items(data: list[dict]) -> list[dict]:\n    \"\"\"\n    Extract and normalize valid items from raw data.\n    \n    :param data: Raw data items\n    :type data: list[dict]\n    :return: Parsed valid items\n    :rtype: list[dict]\n    \"\"\"\n    return [\n        {\n            'id': item['id'],\n            'value': float(item['value']),\n            'category': item.get('category', 'unknown')\n        }\n        for item in data\n        if item.get('valid')\n    ]\n\n\ndef calculate_statistics(items: list[dict]) -> dict:\n    \"\"\"\n    Calculate total and average from items.\n    \n    :param items: Parsed items\n    :type items: list[dict]\n    :return: Statistics dictionary\n    :rtype: dict\n    \"\"\"\n    values = [item['value'] for item in items]\n    total = sum(values)\n    average = total / len(values) if values else 0\n    return {'total': total, 'average': average}\n\n\ndef group_by_category(items: list[dict]) -> dict[str, list[float]]:\n    \"\"\"\n    Group item values by category.\n    \n    :param items: Parsed items\n    :type items: list[dict]\n    :return: Values grouped by category\n    :rtype: dict[str, list[float]]\n    \"\"\"\n    by_category = {}\n    for item in items:\n        category = item['category']\n        by_category.setdefault(category, []).append(item['value'])\n    return by_category\n\n\ndef format_report(stats: dict, by_category: dict[str, list[float]]) -> str:\n    \"\"\"\n    Format statistics into a readable report.\n    \n    :param stats: Overall statistics\n    :type stats: dict\n    :param by_category: Values by category\n    :type by_category: dict[str, list[float]]\n    :return: Formatted report\n    :rtype: str\n    \"\"\"\n    lines = [\n        f\"Total: {stats['total']}\",\n        f\"Average: {stats['average']:.2f}\",\n        \"\\nBy Category:\"\n    ]\n    \n    for category, values in by_category.items():\n        avg = sum(values) / len(values)\n        lines.append(f\"  {category}: {avg:.2f}\")\n    \n    return '\\n'.join(lines)\n\n\ndef generate_report(data: list[dict]) -> str:\n    \"\"\"\n    Generate a comprehensive report from data.\n    \n    :param data: Raw data items\n    :type data: list[dict]\n    :return: Formatted report\n    :rtype: str\n    \"\"\"\n    items = parse_valid_items(data)\n    stats = calculate_statistics(items)\n    by_category = group_by_category(items)\n    return format_report(stats, by_category)

### Aprendizaje Clave\n\nLas funciones deben ser pequeñas. Idealmente, no más de 20 líneas. Si una función es más larga, probablemente puede dividirse en funciones más pequeñas y enfocadas.\n\n**Referencia oficial:** [Clean Code by Robert C. Martin](https://www.oreilly.com/library/view/clean-code-a/9780136083238/)

## 3. Limitar el Número de Parámetros\n\n### El Problema que Resuelve\n\nLas funciones con muchos parámetros son difíciles de entender, llamar y probar. Cada parámetro adicional aumenta la complejidad exponencialmente.

In [ ]:
# BAD: Too many parameters\ndef create_user(first_name, last_name, email, phone, address, city, \n                state, zip_code, country, birth_date, gender):\n    \"\"\"Create a new user with all details.\"\"\"\n    # Implementation\n    pass

In [ ]:
# GOOD: Use a data class to group related parameters\nfrom dataclasses import dataclass\nfrom datetime import date\n\n\n@dataclass\nclass UserProfile:\n    \"\"\"User profile information.\"\"\"\n    first_name: str\n    last_name: str\n    email: str\n    phone: str\n    \n\n@dataclass\nclass Address:\n    \"\"\"User address information.\"\"\"\n    street: str\n    city: str\n    state: str\n    zip_code: str\n    country: str\n    \n\n@dataclass\nclass PersonalInfo:\n    \"\"\"Personal information.\"\"\"\n    birth_date: date\n    gender: str\n\n\ndef create_user(profile: UserProfile, address: Address, \n                personal_info: PersonalInfo) -> dict:\n    \"\"\"\n    Create a new user with organized information.\n    \n    :param profile: User profile data\n    :type profile: UserProfile\n    :param address: Address data\n    :type address: Address\n    :param personal_info: Personal information\n    :type personal_info: PersonalInfo\n    :return: Created user data\n    :rtype: dict\n    \"\"\"\n    # Implementation\n    return {\n        'profile': profile,\n        'address': address,\n        'personal_info': personal_info\n    }

### Aprendizaje Clave\n\nLimita las funciones a 3 parámetros o menos. Si necesitas más, agrupa parámetros relacionados en objetos o usa **kwargs para opciones.\n\n**Referencia oficial:** [Python dataclasses](https://docs.python.org/3/library/dataclasses.html)

## 4. Niveles de Abstracción\n\n### El Problema que Resuelve\n\nMezclar diferentes niveles de abstracción en una función hace que sea difícil de leer y entender. El lector debe saltar entre detalles de bajo nivel y conceptos de alto nivel.

In [ ]:
# BAD: Mixed abstraction levels\ndef process_orders(orders):\n    \"\"\"Process customer orders.\"\"\"\n    for order in orders:\n        # High-level concept\n        validate_order(order)\n        \n        # Low-level details mixed in\n        total = 0\n        for item in order['items']:\n            price = item['price']\n            quantity = item['quantity']\n            discount = item.get('discount', 0)\n            total += price * quantity * (1 - discount)\n        \n        # High-level concept again\n        charge_customer(order['customer_id'], total)\n        send_confirmation(order)

In [ ]:
# GOOD: Consistent abstraction level\ndef calculate_order_total(order: dict) -> float:\n    \"\"\"\n    Calculate total price for an order.\n    \n    :param order: Order data\n    :type order: dict\n    :return: Total price\n    :rtype: float\n    \"\"\"\n    return sum(\n        item['price'] * item['quantity'] * (1 - item.get('discount', 0))\n        for item in order['items']\n    )\n\n\ndef process_single_order(order: dict) -> None:\n    \"\"\"\n    Process a single customer order.\n    \n    :param order: Order data\n    :type order: dict\n    \"\"\"\n    validate_order(order)\n    total = calculate_order_total(order)\n    charge_customer(order['customer_id'], total)\n    send_confirmation(order)\n\n\ndef process_orders(orders: list[dict]) -> None:\n    \"\"\"\n    Process multiple customer orders.\n    \n    :param orders: List of orders\n    :type orders: list[dict]\n    \"\"\"\n    for order in orders:\n        process_single_order(order)

### Aprendizaje Clave\n\nMantén un nivel de abstracción consistente en cada función. Las operaciones de alto nivel deben llamar a funciones de nivel medio, que a su vez llaman a funciones de bajo nivel.\n\n**Referencia oficial:** [The Stepdown Rule - Clean Code](https://www.oreilly.com/library/view/clean-code-a/9780136083238/)

## Ejercicios Prácticos\n\nAhora es tu turno de practicar escribiendo funciones limpias. Completa los ejercicios en el archivo `exercises/01_clean_functions.py` y verifica tus soluciones con `pytest tests/test_01_clean_functions.py`.

### Ejercicio 1: Refactorizar Función con Múltiples Responsabilidades\n\nRefactoriza la siguiente función que hace demasiadas cosas en funciones más pequeñas y enfocadas.

In [ ]:
# Ver exercises/01_clean_functions.py para el ejercicio completo

### Ejercicio 2: Reducir Parámetros\n\nRefactoriza una función con muchos parámetros usando dataclasses o diccionarios.

In [ ]:
# Ver exercises/01_clean_functions.py para el ejercicio completo

### Ejercicio 3: Separar Niveles de Abstracción\n\nReorganiza una función que mezcla diferentes niveles de abstracción.

In [ ]:
# Ver exercises/01_clean_functions.py para el ejercicio completo

## Resumen\n\nEn este notebook has aprendido los principios fundamentales para escribir funciones limpias:\n\n1. **Responsabilidad Única**: Cada función debe hacer una cosa y hacerla bien\n2. **Funciones Pequeñas**: Mantén las funciones cortas (idealmente < 20 líneas)\n3. **Pocos Parámetros**: Limita a 3 parámetros o menos, agrupa parámetros relacionados\n4. **Niveles de Abstracción**: Mantén un nivel de abstracción consistente en cada función\n5. **Nombres Descriptivos**: El nombre debe describir claramente lo que hace la función\n\nEstos principios te ayudarán a escribir código más mantenible, testeable y fácil de entender.

## Preguntas de Autoevaluación\n\n### 1. ¿Qué significa que una función tenga una sola responsabilidad?\n\n**Respuesta:** Significa que la función debe tener una sola razón para cambiar. Debe hacer una cosa específica y hacerla bien, sin mezclar múltiples tareas o responsabilidades.\n\n### 2. ¿Cuál es el número ideal de parámetros para una función?\n\n**Respuesta:** Idealmente 3 o menos. Si necesitas más parámetros, considera agrupar parámetros relacionados en objetos (dataclasses, namedtuples) o usar **kwargs para opciones.\n\n### 3. ¿Cómo identificas que una función es demasiado larga?\n\n**Respuesta:** Si la función tiene más de 20 líneas, si hace múltiples cosas que podrías describir con \"y\" u \"o\", o si no puedes entender qué hace sin leer todo el código, probablemente es demasiado larga.\n\n### 4. ¿Qué problema resuelve mantener niveles de abstracción consistentes?\n\n**Respuesta:** Evita que el lector tenga que saltar entre detalles de bajo nivel y conceptos de alto nivel, haciendo el código más fácil de leer y entender como una narrativa coherente.\n\n### 5. ¿Cuándo deberías dividir una función en funciones más pequeñas?\n\n**Respuesta:** Cuando la función hace múltiples cosas, cuando es difícil de testear, cuando tiene múltiples niveles de indentación, o cuando no puedes darle un nombre descriptivo sin usar \"y\" u \"o\".\n\n### 6. ¿Qué ventajas tiene escribir funciones pequeñas y enfocadas?\n\n**Respuesta:** Son más fáciles de entender, testear, reutilizar y mantener. También facilitan la identificación de bugs y la refactorización del código.

## Recursos y Referencias Oficiales\n\n### Documentación Oficial\n- **[PEP 8 - Style Guide for Python Code](https://peps.python.org/pep-0008/)**: Guía oficial de estilo para Python\n  - Convenciones de nomenclatura y estructura de código\n\n- **[Python dataclasses](https://docs.python.org/3/library/dataclasses.html)**: Documentación oficial de dataclasses\n  - Útil para agrupar parámetros relacionados\n\n### Libros y Recursos\n- **[Clean Code by Robert C. Martin](https://www.oreilly.com/library/view/clean-code-a/9780136083238/)**: Referencia fundamental sobre código limpio\n  - Capítulo 3: Functions\n\n- **[Refactoring by Martin Fowler](https://refactoring.com/)**: Técnicas de refactorización\n  - Extract Function, Introduce Parameter Object\n\n- **[Python Clean Code](https://testdriven.io/blog/clean-code-python/)**: Guía de código limpio específica para Python\n  - Ejemplos prácticos en Python\n\n### Herramientas\n- **[Ruff](https://docs.astral.sh/ruff/)**: Linter y formateador rápido para Python\n  - Detecta funciones complejas y problemas de estilo\n\n- **[pylint](https://pylint.pycqa.org/)**: Analizador de código Python\n  - Detecta funciones con demasiados parámetros o líneas\n\n### Notas Importantes\n- Todos los enlaces están actualizados a partir de 2025\n- Se recomienda revisar la documentación oficial regularmente\n- Practica estos principios en tu código diario para internalizarlos